In [32]:
!pip install pymoo
!pip install deap

In [33]:
from math import factorial
import random

# import matplotlib.pyplot as plt
import numpy as np
import pymoo.factory
from pymoo.problems import get_problem

import copy

from deap import algorithms
from deap import base
from deap.benchmarks.tools import igd
from deap import creator
from deap import tools

In [37]:
# Problem definition
PROBLEM = "dtlz2"
NOBJ = 3 # 目的関数
K = 10 # 追加のパラメータ
NDIM = NOBJ + K - 1 #次元数
P = 12 # ref_pointsで生成する参照点の数
H = factorial(NOBJ + P - 1) / (factorial(P) * factorial(NOBJ - 1))
BOUND_LOW, BOUND_UP = 0.0, 1.0
#problem = pymoo.factory.get_problem(PROBLEM, n_var=NDIM, n_obj=NOBJ)
problem = get_problem(PROBLEM, n_var=NDIM, n_obj=NOBJ)
##

# Algorithm parameters
MU = int(H + (4 - H % 4)) # 個体数
NGEN = 500 # 何世代まで更新するか
CXPB = 1.0 # 交叉（こうさ）率
MUTPB = 1.0 # 変異率
##

# Create uniform reference point
ref_points = tools.uniform_reference_points(NOBJ, P)

# Create classes
creator.create("FitnessMin", base.Fitness, weights=(-1.0,) * NOBJ)
# base.Fitness: 個体の適応度を表すクラス

creator.create("Individual", list, fitness=creator.FitnessMin)
##


# Toolbox initialization
def uniform(low, up, size=None):
    try:
        return [random.uniform(a, b) for a, b in zip(low, up)]
    except TypeError:
        return [random.uniform(a, b) for a, b in zip([low] * size, [up] * size)]

def eva(ind):
    a = (ind[0] + ind[1] + ind[2] + ind[3] + ind[4] + ind[5] + ind[6] + ind[7] + ind[8] + ind[9] + ind[10] + ind[11]) / 12
    b = (ind[0] + ind[1] + ind[2] + ind[3] +ind[4] + ind[5]) / 6
    c = (ind[6] + ind[7] + ind[8] + ind[9] + ind[10] + ind[11]) / 6

    return a,b,c

toolbox = base.Toolbox()
toolbox.register("attr_float", uniform, BOUND_LOW, BOUND_UP, NDIM)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_float)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", problem.evaluate, return_values_of=["F"])
# toolbox.register("evaluate", eva)
toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=BOUND_LOW, up=BOUND_UP, eta=30.0)
toolbox.register("mutate", tools.mutPolynomialBounded, low=BOUND_LOW, up=BOUND_UP, eta=20.0, indpb=1.0/NDIM)
toolbox.register("select", tools.selNSGA3, ref_points=ref_points)
##

def main(seed=None):
    random.seed(seed)

    # Initialize statistics object
    # stats = tools.Statistics(lambda ind: ind.fitness.values)
    # stats.register("avg", numpy.mean, axis=0)
    # stats.register("std", numpy.std, axis=0)
    # stats.register("min", numpy.min, axis=0)
    # stats.register("max", numpy.max, axis=0)

    # logbook = tools.Logbook()
    # logbook.header = "gen", "evals", "std", "min", "avg", "max"

    pop = toolbox.population(n=MU)

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in pop if not ind.fitness.valid]
    print("toolbox.evaluate",toolbox.evaluate)
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    print("invalid_ind:",invalid_ind)
    print("fitnesses:",fitnesses)
    for ind, fit in zip(invalid_ind, fitnesses):
        # print("ind:",ind,"fitnesses:",fit)
        ind.fitness.values = fit

    print("len:", len(pop))
    for i in range(len(pop)):
        print("初期pop:",pop[i])
        print("fitness:",pop[i].fitness.values)
    print("type:",type(pop[0]))


    # # Compile statistics about the population
    # record = stats.compile(pop)
    # logbook.record(gen=0, evals=len(invalid_ind), **record)
    # print(logbook.stream)

    # Begin the generational process
    for gen in range(1, NGEN):
        offspring = algorithms.varAnd(pop, toolbox, CXPB, MUTPB)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Select the next generation population from parents and offspring
        if(gen == 499):
            for i in range(len(pop)):
                print("選択前pop:",pop[i])
                print("fitness:",pop[i].fitness.values)
            for i in range(len(offspring)):
                print("選択前offspring:",offspring[i])
                print("fitness:",offspring[i].fitness.values)

        pop = toolbox.select(pop + offspring, MU)

        if(gen == 499):
            print("-------------------")
            print("-------------------")
            print("-------------------")
            print("-------------------")
            pop.sort()
            for i in range(len(pop)):
                print("選択後pop:",pop[i])
                print("fitness:",pop[i].fitness.values)

        # # Compile statistics about the new population
        # record = stats.compile(pop)
        # logbook.record(gen=gen, evals=len(invalid_ind), **record)
        # print(logbook.stream)

    return pop ##, logbook


if __name__ == "__main__":
    pop = main()
    print("Success")
    # pop_fit = numpy.array([ind.fitness.values for ind in pop])

    # pf = problem.pareto_front(ref_points)
    # print(igd(pop_fit, pf))

    # import matplotlib.pyplot as plt
    # import mpl_toolkits.mplot3d as Axes3d

    # fig = plt.figure(figsize=(7, 7))
    # ax = fig.add_subplot(111, projection="3d")

    # p = numpy.array([ind.fitness.values for ind in pop])
    # ax.scatter(p[:, 0], p[:, 1], p[:, 2], marker="o", s=24, label="Final Population")

    # ax.scatter(pf[:, 0], pf[:, 1], pf[:, 2], marker="x", c="k", s=32, label="Ideal Pareto Front")

    # ref_points = tools.uniform_reference_points(NOBJ, P)

    # ax.scatter(ref_points[:, 0], ref_points[:, 1], ref_points[:, 2], marker="o", s=24, label="Reference Points")

    # ax.view_init(elev=11, azim=-25)
    # ax.autoscale(tight=True)
    # plt.legend()
    # plt.tight_layout()
    # plt.savefig("nsga3.png")

toolbox.evaluate functools.partial(<bound method Problem.evaluate of <pymoo.problems.many.dtlz.DTLZ2 object at 0x7d25c84eb190>>, return_values_of=['F'])
invalid_ind: [[0.6858446180999834, 0.6859080454417887, 0.025417434183592436, 0.8032384980357452, 0.41714476003611367, 0.18214138934977908, 0.07010448376683565, 0.24783126076091289, 0.9782991564657505, 0.813690468161799, 0.39991570532152665, 0.21893217308088508], [0.15841021776928232, 0.2289677508900303, 0.22672908275266124, 0.15763676516189384, 0.7092782188324044, 0.0879373968505377, 0.650087261759311, 0.6204137116691603, 0.2181282530114994, 0.04452142766399336, 0.5937855741280464, 0.982491295302079], [0.5700554190062537, 0.11481863760446975, 0.591852229810822, 0.7699596393388696, 0.31806527395950934, 0.5689316825823346, 0.6476498094853109, 0.5458310564666947, 0.8126214754465901, 0.414605817236666, 0.167296401345714, 0.6010084872267154], [0.6198677410192546, 0.7683164196819042, 0.9699163820013932, 0.8040475912282881, 0.4289107068761959

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


TypeError: list indices must be integers or slices, not tuple